Reading Code


In [ ]:
 
import os
import csv
import serial
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def read_and_store_serial(port, baud_rate, num_samples=10, csv_folder='C:\\Users\\shukl\\Desktop\\IISc Work\\VINAYAK 2023 24\\Vinayak_pk\\power_detectors_programs\\data'):
    if not os.path.exists(csv_folder):
        os.makedirs(csv_folder)

    ser = serial.Serial(port, baud_rate)

    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    csv_file_path = os.path.join(csv_folder, f'data_{timestamp}.csv')

    header_written = False

    for i in range(num_samples):
        data = ser.readline().decode('utf-8', errors='ignore').strip()
        try:
            
            values = [float(val.split(':')[1]) for val in data.split(',') if ':' in val]
            print("Received values:", values)

           
            if not header_written:
                save_to_csv(values, csv_file_path, header=True)
                header_written = True
            else:
                save_to_csv(values, csv_file_path, header=False)

        except ValueError:
            print(f"Skipping non-float values: {data}")

    ser.close()

def save_to_csv(data_list, csv_file_path, header=False):
    with open(csv_file_path, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile, quoting=csv.QUOTE_NONNUMERIC, escapechar='\\')

        # Write header if it hasn't been written
        if header:
            csv_writer.writerow(['SUM', 'DIFF'])

        csv_writer.writerow(data_list)

# Create a separate file of mean value in the following directory
def save_means_to_csv(mean_sum, mean_diff, means_csv_path='C:\\Users\\shukl\\Desktop\\IISc Work\\VINAYAK 2023 24\\Vinayak_pk\\power_detectors_programs\\mean_values.csv'):
    file_exists = os.path.isfile(means_csv_path)

    with open(means_csv_path, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile, quoting=csv.QUOTE_NONNUMERIC, escapechar='\\')

        # Write header if file does not exist
        if not file_exists:
            csv_writer.writerow(['Timestamp', 'Mean SUM', 'Mean DIFF'])

        csv_writer.writerow([datetime.now().strftime("%Y-%m-%d %H:%M:%S"), mean_sum, mean_diff])

if __name__ == "__main__":
    serial_port = "COM9"  # Change this to your serial port
    baud_rate = 115200      # Change this to your baud rate
    num_samples = 560      #You can change this points as per as your requirement

    read_and_store_serial(serial_port, baud_rate, num_samples)

    # Path to the folder where CSV files are stored
    csv_folder = 'C:\\Users\\shukl\\Desktop\\IISc Work\\VINAYAK 2023 24\\Vinayak_pk\\power_detectors_programs\\data'

    # Get the list of CSV files in the directory
    csv_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]

    # Sort the files by modification time in descending order
    csv_files.sort(key=lambda f: os.path.getmtime(os.path.join(csv_folder, f)), reverse=True)

    # Use the latest CSV file
    latest_csv_file = csv_files[0]
    csv_file_path = os.path.join(csv_folder, latest_csv_file)

    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)

    
    print(df)

    #  mean values-
    mean_sum = df["SUM"].mean()
    mean_diff = df["DIFF"].mean()
    print("Mean Value of Sum:", mean_sum)
    print("Mean Value of Difference:", mean_diff)

    # Save mean values to CSV
    save_means_to_csv(mean_sum, mean_diff)


Plotting


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd


df = pd.read_csv('C:\\Users\\shukl\\Desktop\\IISc Work\\VINAYAK 2023 24\\Vinayak_pk\\power_detectors_programs\\detector_2.csv')
df2 = pd.read_csv('C:\\Users\\shukl\\Desktop\\IISc Work\\VINAYAK 2023 24\\Vinayak_pk\\power_detectors_programs\\detector_3.csv')
df3 = pd.read_csv('C:\\Users\\shukl\\Desktop\\IISc Work\\VINAYAK 2023 24\\Vinayak_pk\\power_detectors_programs\\detector_15.csv')

# Extract the 'Mean SUM' columns
voltage_2 = df['Mean SUM']
voltage_3 = df2['Mean SUM']
voltage_1 = df3['Mean SUM 1']
voltage_5 = df3['Mean SUM 5']

array1 = np.arange(-49, -17, 2)
array2 = np.arange(-18, 2, 2)
power = np.concatenate((array1, array2))


new_df = pd.DataFrame({'Power': power, 'Voltage 2': voltage_2, 'Voltage 3': voltage_3, 'Voltage 1': voltage_1, 'Voltage 5': voltage_5})

# Saving the new DataFrame to a CSV file
new_df.to_csv('C:\\Users\\shukl\\Desktop\\IISc Work\\VINAYAK 2023 24\\Vinayak_pk\\power_detectors_programs\\combined_voltages.csv', index=False)

# Set up the figure and axes
fig, ax = plt.subplots(figsize=(12, 8))

# power vs voltage for both voltages
ax.plot(new_df['Power'], new_df['Voltage 2'], label='Voltage 2(Male-Female Port)', color='#1f77b4', linewidth=2)
ax.plot(new_df['Power'], new_df['Voltage 3'], label='Voltage 3(Male-Female Port)', color='#ff7f0e', linewidth=2)
ax.plot(new_df['Power'], new_df['Voltage 1'], label='Voltage 1(USB Port)', color='#2ca02c', linewidth=2)
ax.plot(new_df['Power'], new_df['Voltage 5'], label='Voltage 5(USB Port)', color='#d62728', linewidth=2)


ax.set_xlabel('Power (dBm)', fontsize=14)
ax.set_ylabel('Voltage (V)', fontsize=14)
ax.set_title('Power vs Voltage Plot for All Voltages', fontsize=16, fontweight='bold')


ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.02))
ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=10)

# vertical line at power = -20 dBm
ax.axvline(x=-20, color='#7f7f7f', linestyle='--', linewidth=1)

# Annotate the vertical line
ax.annotate("with 20 dBm attenuator", xy=(-22, 0), xytext=(-30, 0.5),
            arrowprops=dict(arrowstyle='->', color='black'), fontsize=10, color='black')
ax.annotate("without attenuator", xy=(-18, 0), xytext=(-10, 0.5),
            arrowprops=dict(arrowstyle='->', color='black'), fontsize=10, color='black')

# Marking each data point
for i, (power, voltage_2, voltage_3, voltage_1, voltage_5) in enumerate(zip(new_df['Power'], new_df['Voltage 2'], new_df['Voltage 3'], new_df['Voltage 1'], new_df['Voltage 5'])):
    ax.scatter(power, voltage_2, s=50, color='#1f77b4', marker='o', label='_nolegend_')
    ax.annotate(f"{voltage_2:.2f}", (power, voltage_2), xytext=(5, 5), textcoords="offset points", fontsize=8)
    ax.scatter(power, voltage_3, s=50, color='#ff7f0e', marker='o', label='_nolegend_')
    ax.annotate(f"{voltage_3:.2f}", (power, voltage_3), xytext=(5, -15), textcoords="offset points", fontsize=8)
    ax.scatter(power, voltage_1, s=50, color='#2ca02c', marker='o', label='_nolegend_')
    ax.annotate(f"{voltage_1:.2f}", (power, voltage_1), xytext=(5, 5), textcoords="offset points", fontsize=8)
    ax.scatter(power, voltage_5, s=50, color='#d62728', marker='o', label='_nolegend_')
    ax.annotate(f"{voltage_5:.2f}", (power, voltage_5), xytext=(5, -15), textcoords="offset points", fontsize=8)


ax.grid(True, which='major', linestyle='-', linewidth=0.5, color='#d0d0d0')
ax.grid(True, which='minor', linestyle='--', linewidth=0.25, color='#e0e0e0')
ax.minorticks_on()


ax.legend(loc='upper left', fontsize=12)

# Adjust the layout
plt.subplots_adjust(left=0.1, right=0.95, top=0.9, bottom=0.1)

# Save the plot
plt.savefig('C:\\Users\\shukl\\Desktop\\IISc Work\\VINAYAK 2023 24\\Vinayak_pk\\power_detectors_programs\\power_vs_voltage_plot.png', dpi=300)

# Show the plot
plt.show()